Our student exam numbers given for this exam 


In [5]:
import numpy as np 
import pandas as pd 

In [6]:
def importCSV():
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    all_dfs = []

    for month in months:
        for year in range(2014, 2025):
            csv_string = f"Term paper/data/SCE-{month}-{year}.csv" 
            try:
                csv_file = pd.read_csv(csv_string, sep=";")
                all_dfs.append(csv_file)
            except FileNotFoundError:
                print(f"Filen {csv_string} finnes ikke.")

    return  pd.concat(all_dfs)
df = importCSV()

In [7]:

unique_persons = df["userid"].nunique()

print(f"unique persons: {unique_persons}")
print(f"number of rows: {df.shape[0]}")

print(f"Number of unique survey waves: {df['wid'].nunique()}")

earliest_date = df['date'].min()
print(f"earliest_date: {earliest_date}")

latest_date = df['date'].max()
print(f"latest date: {latest_date}")

df.head()


unique persons: 21666
number of rows: 165924
Number of unique survey waves: 132
earliest_date: 2014-01-02
latest date: 2024-12-31


,userid,wid,date,weight,female,educ,age,hispanic,black,couple,...,num_lit_q3,num_lit_q3_correct,num_lit_q5,num_lit_q5_correct,num_lit_q6,num_lit_q6_correct,num_lit_q8,num_lit_q8_correct,num_lit_q9,num_lit_q9_correct
0,70016297,201401,2014-01-07,1.0,0.0,4.0,29.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70016329,201401,2014-01-02,4.5,1.0,2.0,43.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70016366,201401,2014-01-07,0.6,0.0,4.0,30.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,70016375,201401,2014-01-16,0.7,1.0,4.0,28.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,70016445,201401,2014-01-19,0.5,0.0,4.0,47.0,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# ---- Part 2: Data preprocessing ----
# Assumes you already have: df = importCSV()

# 0) Basic hygiene
df = df.copy()
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

# --- Helpers for neat reporting ---
def sample_stats(_df, title="Sample stats"):
    n_obs = len(_df)
    n_indiv = _df["userid"].nunique() if "userid" in _df.columns else np.nan
    n_waves = _df["date"].dt.to_period("M").nunique() if "date" in _df.columns else np.nan
    dmin = _df["date"].min() if "date" in _df.columns else None
    dmax = _df["date"].max() if "date" in _df.columns else None
    print(f"\n== {title} ==")
    print(f"Observations: {n_obs:,}")
    print(f"Unique individuals: {n_indiv:,}" if pd.notna(n_indiv) else "Unique individuals: N/A")
    print(f"Survey waves (months): {n_waves:,}" if pd.notna(n_waves) else "Survey waves (months): N/A")
    if dmin is not None:
        print(f"Date range: {dmin.strftime('%Y-%m')} to {dmax.strftime('%Y-%m')}")

# Snapshot before cleaning
sample_stats(df, "Before cleaning")

# 1) Forward-fill numeracy (per individual) from the first observation
#    The term paper says: forward-fill the *_correct variables only (asked at first entry).
num_cols = [c for c in df.columns if c.endswith("_correct") and c.startswith("num_lit_")]
if num_cols:
    first_vals = df.groupby("userid", dropna=False)[num_cols].transform("first")
    # Only fill where current is missing
    before_missing_num = df[num_cols].isna().sum().sum()
    df[num_cols] = df[num_cols].fillna(first_vals)
    after_missing_num = df[num_cols].isna().sum().sum()
    print(f"\nFilled numeracy (from first obs per userid): "
          f"{before_missing_num - after_missing_num} cells filled")

# 2) Drop rows with missing required fields
#    Demographics: female (indicator), age, educ
#    Expectations: inflation, house_price_change, prob_stocks_up
#    Numeracy: all *_correct columns (after forward fill)
required_cols = ["female", "age", "educ", "inflation", "house_price_change", "prob_stocks_up"] + num_cols
required_cols = [c for c in required_cols if c in df.columns]  # keep only those that exist

before = len(df)
df = df.dropna(subset=required_cols)
dropped_missing = before - len(df)
print(f"\nDropped rows with missing required values: {dropped_missing:,}")

# 3) Drop outliers using 0.1th and 99.9th percentiles for each expectations variable
exp_vars = [c for c in ["inflation", "house_price_change", "prob_stocks_up"] if c in df.columns]
dropped_outliers_total = 0
for col in exp_vars:
    q_low = df[col].quantile(0.001)
    q_high = df[col].quantile(0.999)
    mask_keep = (df[col] >= q_low) & (df[col] <= q_high)
    dropped = (~mask_keep).sum()
    df = df.loc[mask_keep].copy()
    dropped_outliers_total += int(dropped)
    print(f"Dropped outliers for {col}: {int(dropped):,} "
          f"(kept [{q_low:.3f}, {q_high:.3f}])")

print(f"Total outliers dropped: {dropped_outliers_total:,}")

# 4) Create 'college' = 1 if educ >= 4 (Bachelor’s or higher), else 0
#    (educ is ordinal: 1=no HS/GED, 2=HS/GED, 3=some college/assoc., 4=bachelor’s+)
if "educ" in df.columns:
    df["college"] = (df["educ"] >= 4).astype(int)

# 5) Sum correct numeracy answers per row and create 'num_lit_high'
if num_cols:
    df["num_correct"] = df[num_cols].sum(axis=1)
    med = df["num_correct"].median()
    # "more correct responses than the median"
    df["num_lit_high"] = (df["num_correct"] > med).astype(int)

# 6) Final sample stats (same structure as Part 1)
sample_stats(df, "After cleaning (final dataset)")

# Optional: quick sanity checks
print("\nMissing values (selected):")
for c in ["female", "age", "educ", "inflation", "house_price_change", "prob_stocks_up", "college", "num_lit_high"]:
    if c in df.columns:
        print(f"{c:<22} {df[c].isna().sum():>6}")

# Keep the cleaned DataFrame for later parts
df_clean = df



== Before cleaning ==
Observations: 165,924
Unique individuals: 21,666
Survey waves (months): 132
Date range: 2014-01 to 2024-12

Filled numeracy (from first obs per userid): 936645 cells filled

Dropped rows with missing required values: 28,348
Dropped outliers for inflation: 119 (kept [-75.000, 100.000])
Dropped outliers for house_price_change: 179 (kept [-50.000, 100.000])
Dropped outliers for prob_stocks_up: 0 (kept [0.000, 100.000])
Total outliers dropped: 298

== After cleaning (final dataset) ==
Observations: 137,278
Unique individuals: 17,701
Survey waves (months): 117
Date range: 2015-04 to 2024-12

Missing values (selected):
female                      0
age                         0
educ                        0
inflation                   0
house_price_change          0
prob_stocks_up              0
college                     0
num_lit_high                0
